In [1]:
import torch
import src

In [2]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
device = torch.device("cuda" if use_cuda else "cpu")
print("Device: ", device)

__CUDNN VERSION: 7605
__Number CUDA Devices: 1
__CUDA Device Name: GeForce GTX 1060
__CUDA Device Total Memory [GB]: 6.373572608
Device:  cuda


In [3]:
from sklearn.preprocessing import LabelEncoder
from src.utils import ucr_2018, data

train_df, test_df = ucr_2018.get_dataset("ECG200")
X_train, y_train = data.x_y_split(train_df, y_col=0)
X_test, y_test = data.x_y_split(test_df, y_col=0)

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [4]:
params = {
    'batch_size': 10,
    'shuffle': True
}

train = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.values).unsqueeze(1).float().to(device),
    torch.tensor(y_train).to(device)
)
train_loader = torch.utils.data.DataLoader(train, **params)

test = torch.utils.data.TensorDataset(
    torch.from_numpy(X_test.values).unsqueeze(1).float().to(device),
    torch.tensor(y_test).to(device)
)
test_loader = torch.utils.data.DataLoader(test, **params)

In [5]:
for tmp, tmp2 in train_loader:
    print(tmp.size())
    break

torch.Size([10, 1, 96])


In [6]:
from src.cnn_classifier import CNNClassifier

epochs = 100
model = CNNClassifier(
        lr=1e-4,
        Optimizer=torch.optim.Adam,
        device=device
).to(device)
model.train(train_loader=train_loader, validation_loader=test_loader,
            epochs=epochs, model_name="CNN_classifier", logdir="logs",
            verbose=1)

Epoch 0, dataset train: Loss - 0.696952, F1-score - 0.465116
Epoch 0, dataset validation: Loss - 0.693359, F1-score - 0.705882
Epoch 1, dataset train: Loss - 0.691537, F1-score - 0.750000
Epoch 1, dataset validation: Loss - 0.688805, F1-score - 0.773006
Epoch 2, dataset train: Loss - 0.686039, F1-score - 0.816568
Epoch 2, dataset validation: Loss - 0.684315, F1-score - 0.780488
Epoch 3, dataset train: Loss - 0.680959, F1-score - 0.816568
Epoch 3, dataset validation: Loss - 0.679642, F1-score - 0.780488
Epoch 4, dataset train: Loss - 0.675519, F1-score - 0.816568
Epoch 4, dataset validation: Loss - 0.675066, F1-score - 0.780488
Epoch 5, dataset train: Loss - 0.669870, F1-score - 0.816568
Epoch 5, dataset validation: Loss - 0.670390, F1-score - 0.780488
Epoch 6, dataset train: Loss - 0.664493, F1-score - 0.816568
Epoch 6, dataset validation: Loss - 0.665610, F1-score - 0.780488
Epoch 7, dataset train: Loss - 0.659719, F1-score - 0.816568
Epoch 7, dataset validation: Loss - 0.660682, F1-s

In [7]:
#!rm -rf ./logs/CNN_clasifier